In [1]:
import os
import json
import pprint
import copy

os.chdir('../src/meshflow/')
os.getcwd()

'/Users/kasrakeshavarz/Documents/github-repos/meshflow/src/meshflow'

In [2]:
# # necessary functions
def deep_merge(d1, d2):
    """
    Recursively merge d2 into d1.
    """
    for key, value in d2.items():
        if key in d1 and isinstance(d1[key], dict) and isinstance(value, dict):
            deep_merge(d1[key], value)  # Recursive merge for nested dictionaries
        else:
            d1[key] = value  # Overwrite or add new key-value pair
    return d1

# CLASS file

In [3]:
case_entry = {
    "centroid_lat": 45,
    "centroid_lon": -115,
    "reference_height_wndspd": 40.0,
    "reference_height_spechum_airtemp": 40.0,
    "reference_height_surface_roughness": 50.0,
    "NL": 12,
    "NM": 5,
}

info_entry = {
    "author": "Kasra Keshavarz",
    "location": "Calgary, Alberta"
}

gru_entry = {
    1: {
        'class': 'needleleaf',
        'fcan': 1,
        'lamx': 2,
        'mid': 'AB Needleleaf'
    },
    2: {
        'class': 'broadleaf',
        'clay1': 20,
        'mid': 'BC Broadleaf'
    },
}

In [4]:
# Source code additions
with open('./templates/default_CLASS_lines.json', 'r') as file:
    gru_lines = json.load(file)

with open('./templates/default_CLASS_types.json', 'r') as file:
    gru_types = json.load(file)

gru_block = {"vars": []}

for gru, params in gru_entry.items():
    d = {}
    for param, param_value in params.items():
        if param == 'class':
            d['veg'] = {'class': param_value}
            continue
        param_line = gru_lines[param]
        param_type = gru_types[param]

        if param_type in d.keys():
            d[param_type].update({f'line{param_line}': {param: param_value}})
        else:
            d[param_type] = {f'line{param_line}': {param: param_value}}
    gru_block['vars'].append(d)

In [5]:
# load the default values for each GRU
with open('./templates/default_CLASS_parameters.json', 'r') as file:
    data = json.load(file)
with open('./templates/default_CLASS_heading.json', 'r') as file:
    info = json.load(file)
with open('./templates/default_CLASS_case.json', 'r') as file:
    case = json.load(file)

# populate new dictionary for GRU blocks
populating_list = []

# deep update GRU blocks
for idx, block in enumerate(gru_block['vars']):
    new_data = copy.deepcopy(data)

    # deep merge
    it = deep_merge(new_data['class_defaults'], block)
    
    # update the block dictionary
    populating_list.append(it)

gru_block.update({'vars': populating_list})

# update case block -> finalized for interface
case_block = deep_merge(case, {"vars": {"case": case_entry}})

# update info block -> finalized for interface
info_block = deep_merge(info, {"vars": info_entry})

# add formats, columns, and comments
new_keys = ['formats', 'comments', 'columns']
for key in new_keys:
    gru_block[key] = data[key]

In [6]:
from jinja2 import Environment, FileSystemLoader

def raise_helper(msg):
    raise Exception(msg)

environment = Environment(
    loader=FileSystemLoader("templates/"),
    trim_blocks=True,
    lstrip_blocks=True,
    line_comment_prefix='##',
)
environment.globals['raise'] = raise_helper

template = environment.get_template("MESH_parameters_CLASS.ini.jinja")

# create content
content = template.render(
    info_block=info_block,
    case_block=case_block,
    gru_block=gru_block,
    variables="vars",
    comments="comments",
    formats="formats",
    columns="columns",
)

print(content)

  MESH Model                                                                 01 TITLE
  Kasra Keshavarz                                                            02 NAME
  Calgary, Alberta                                                           03 PLACE
   45.00   -115.00      40.0      40.0      50.0   -1.0    1    12   5       04 DEGLAT/DEGLON/ZRFM/ZRFH/ZBLD/GC/ILW/NL/NM
   1.000   0.000   0.000   0.000   0.000   2.000   0.000   0.000   0.000     05 5xFCAN/4xLAMX
  -1.300   0.000   0.000   0.000   0.000   1.200   0.000   0.000   0.000     06 5xLNZ0/4xLAMN
   0.045   0.000   0.000   0.000   0.000   4.500   0.000   0.000   0.000     07 5xALVC/4xCMAS
   0.160   0.000   0.000   0.000   0.000   1.090   0.000   0.000   0.000     08 5xALIC/4xROOT
 145.000   0.000   0.000   0.000          36.000   0.000   0.000   0.000     09 4xRSMN/4xQA50
   0.800   0.000   0.000   0.000           1.050   0.000   0.000   0.000     10 4xVPDA/4xVPDB
 100.000   0.000   0.000   0.000           5.000   0.000 

____

# Hydrology file

In [7]:
routing_dict = [
    {
        "r2n": 2.87,
        "flz": 7,
    },
    {
        "r2n": 0.2,
    },
]

gru_dict = [
    {
        '1': {
            "zsnl": 0.3
        }
    },
    {
        'longest_block_2': {
            "zplg": 0.9
        }
    },
    {
        '3': {
            "zplg": 0.438
        }
    },
]

In [8]:
# load the default values for each GRU
with open('./templates/default_hydrology_parameters.json', 'r') as file:
    data = json.load(file)

# components
routing_defaults = data.get('routing')
gru_defaults = data.get('gru')

# deep update routing block
for idx, routing_block in enumerate(routing_dict):
    defaults = copy.deepcopy(routing_defaults)

    # deep merge
    defaults.update(routing_block)

    # update the dict (list of dicts)
    routing_dict[idx].update(defaults)

# deep update gru block
for idx, gru_block in enumerate(gru_dict):
    defaults = copy.deepcopy(gru_defaults)

    # update default values
    for k, v in gru_block.items():
        defaults.update(v)

    # update the dict
    gru_dict[idx][k].update(defaults)

In [9]:
routing_dict

[{'r2n': 2.87, 'flz': 7, 'r1n': 0.02, 'pwr': 2.37},
 {'r2n': 0.2, 'r1n': 0.02, 'pwr': 2.37, 'flz': 0.001}]

In [10]:
gru_dict

[{'1': {'zsnl': 0.3, 'zpls': 0.06, 'zplg': 0.2, 'iwf': 1}},
 {'longest_block_2': {'zplg': 0.9, 'zsnl': 0.05, 'zpls': 0.06, 'iwf': 1}},
 {'3': {'zplg': 0.438, 'zsnl': 0.05, 'zpls': 0.06, 'iwf': 1}}]

In [11]:
# hydrology template file
template_hydrology = environment.get_template("MESH_parameters_hydrology.ini.jinja")

# create content
content_hydrology = template_hydrology.render(
    routing_dict=routing_dict,
    gru_dict=gru_dict,
)

print(content_hydrology)

2.0: MESH Hydrology parameters input file (Version 2.0)
##### Option Flags #####
----#
    0                                                       # Number of option flags
##### Channel routing parameters per river class #####
-------#
4                                                           # Number of channel routing parameters
R2N    2.870    0.200                                       # comment for line
R1N    0.020    0.020                                       # comment for line
PWR    2.370    2.370                                       # comment for line
FLZ    7.000    0.001                                       # comment for line
##### GRU class independent hydrologic parameters #####     # 10comment line 13                                                           | *
-------#                                                    # 11comment line 14                                                           | *
       0                                                    # Num

____

# Input config file

In [12]:
options_dict = {
    "flags": {
        "forcing": {
            "BASINSHORTWAVEFLAG": "RDRS_v2.1_P_FB_SFC",
            "BASINHUMIDITYFLAG": "RDRS_v2.1_P_HU_1.5m",
            "BASINRAINFLAG": "RDRS_v2.1_A_PR0_SFC",
            "BASINPRESFLAG": "RDRS_v2.1_P_P0_SFC",
            "BASINLONGWAVEFLAG": "RDRS_v2.1_P_FI_SFC",
            "BASINWINDFLAG": "RDRS_v2.1_P_UVC_10m",
            "BASINTEMPERATUREFLAG": "RDRS_v2.1_P_TT_1.5m",
            "BASINFORCINGFLAG": {
                "start_date": 19800101,
                "hf": 15,
            },
            "FORCINGLIST": "forcing_files_list",
        },
        "etc": {
            "PBSMFLAG": "on",
            "TIMESTEPFLAG": 60,
        },
    },
    "outputs": {
        "result": "results",
    },
    "dates": {
        "start_year": 1980,
        "start_day": 250,
        "start_hour": 23,
        "end_year": 2020,
        "end_day": 352,
        "end_hour": 3
    },
}

In [13]:
# load the default values for each GRU
with open('./templates/default_input_run_options.json', 'r') as file:
    options = json.load(file)

# deep update the dictionary
options['settings'].update(deep_merge(options['settings'], options_dict))

In [14]:
pprint.pprint(options)

{'columns': {'comments': 57, 'data': 24},
 'comments': {'dates': {'end_date': '#28 Stop year, day, hour, minute  (2017 '
                                    '274   0   0)             | 4I4',
                        'start_date': '#27 Start year, day, hour, minute (2000 '
                                      '275   0   0)             | 4I4'},
              'flags': {'class': {'IALC': '#11 CLASS Input Canopy Albedo '
                                          'Override          | A20, I4',
                                  'IALG': '#13 CLASS Input Soil Albedo '
                                          'Override            | A20, I4',
                                  'IALS': '#12 CLASS Input Snow Albedo '
                                          'Override            | A20, I4',
                                  'IDISP': '#02 Vegetation Displacement Height '
                                           'Calculation  | A20, I4',
                                  'IHGT': '#10 CLASS Input Ve

In [15]:
# options template file
template_options = environment.get_template("MESH_input_run_options.ini.jinja")

# create content
content_options = template_options.render(
    options_dict = options,
)

print(content_options)

MESH input run options file                             # comment line 1                                | * 
##### Control Flags #####                               # comment line 2                                | * 
----#                                                   # comment line 3                                | * 
   42                                                   # Number of control flags                       | I5
BASINFORCINGFLAG      nc_subbasin start_date=19800101 hf=15 
BASINSHORTWAVEFLAG    RDRS_v2.1_P_FB_SFC
BASINHUMIDITYFLAG     RDRS_v2.1_P_HU_1.5m
BASINRAINFLAG         RDRS_v2.1_A_PR0_SFC
BASINPRESFLAG         RDRS_v2.1_P_P0_SFC
BASINLONGWAVEFLAG     RDRS_v2.1_P_FI_SFC
BASINWINDFLAG         RDRS_v2.1_P_UVC_10m
BASINTEMPERATUREFLAG  RDRS_v2.1_P_TT_1.5m
FORCINGLIST           forcing_files_list
IDISP                 0                                 #02 Vegetation Displacement Height Calculation  | A20, I4
IZREF                 1                                 #